## Spiral Report crawler

Imperial College London has an open access repository, called Spiral. 
Spiral hosts the reports with more of the metadata exposed than is exposed in the public website access for the reports. Trying to pull the metadata from the public website links requires multiple pages with multiple levels of parsing. For the reports, it's easier to use the data pulled from spiral

In [1]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

Create the Curation Object (ie- the curatedBy object)

In [2]:
def create_curationObject():
    now = datetime.now()
    curatedBy = {
    "@type": "Organization",
    'identifier': 'imperialcollege',
    'url': 'http://www.imperial.ac.uk/mrc-global-infectious-disease-analysis/covid-19/covid-19-reports/',
    "name": "MRC Centre for Global Infectious Disease Analysis",
    "affiliation": ["Imperial College London"],
    "curationDate":now.strftime("%Y-%m-%d")
    }    
    return(curatedBy)

Query Spiral for COVID-19 reports and parse the result into a list of urls of the individual report pages

In [13]:
def get_report_links(reports_url):
    recordlist = requests.get(reports_url)
    spiralbase = "https://spiral.imperial.ac.uk"
    parsedrecordlist = BeautifulSoup(recordlist.text, "html.parser")
    urlstable = parsedrecordlist.findAll("table")[0]
    urlstublist = urlstable.findAll("a")
    url_list = []
    for eachlink in urlstublist:
        tmpurl = spiralbase+eachlink.get("href")
        url_list.append(tmpurl)
    return(url_list)

Since we're dealing with html files, create a function to get content for a specific tag

In [4]:
def get_meta_content(metacontentfield):
    if len(metacontentfield) == 1:
        metacontentlist = metacontentfield[0].get("content")
    else:
        metacontentlist = []
        for eachitem in metacontentfield:
            metaitem = eachitem.get("content")
            metacontentlist.append(metaitem)
    return(metacontentlist)   

Pull content from the appropriate meta data tags and format it

In [5]:
def transform_pub_meta(soupobject):
    urlfield = soupobject.findAll("meta", {"name":"citation_pdf_url"})
    url = get_meta_content(urlfield)
    titlefield = soupobject.findAll("meta", {"name":"citation_title"})
    title = get_meta_content(titlefield)
    datePublishedfield = soupobject.findAll("meta", {"name":"citation_date"})
    datePublished = get_meta_content(datePublishedfield)
    abstractfield = soupobject.findAll("meta", {"name":"DCTERMS.abstract"})
    abstract = get_meta_content(abstractfield)
    defaultidurlfield = soupobject.findAll("meta", {"scheme":"DCTERMS.URI"})
    defaultid = get_meta_content(defaultidurlfield)
    tmpdict = {
        "@context": {
        "schema": "http://schema.org/",
        "outbreak": "https://discovery.biothings.io/view/outbreak/"
        },
        "@type": "Publication",
        "journalName": "Imperial College London",
        "journalNameAbbreviation": "imperialcollege",
        "publicationType": "Report", 
        "abstract":abstract,
        "name":title,
        "datePublished":datePublished,
        "url":url,
        "identifier":defaultid
    }
    keywordsfield = soupobject.findAll("meta", {"name":"DC.subject"})
    if len(keywordsfield)>0:
        keywordsobject = get_meta_content(keywordsfield)
        tmpdict["keywords"] = keywordsobject

    licensefield = soupobject.findAll("meta", {"name":"DC.rights"})
    if len(licensefield)>0:
        license = get_meta_content(licensefield)
        tmpdict["license"] = license
        
    identifiersfield = soupobject.findAll("meta", {"name":"DC.identifier"})
    for eachitem in identifiersfield:
        eachitemcontent = eachitem.get("content")
        if "doi" in eachitemcontent:
            doi = eachitemcontent.replace("https://doi.org/","")
            tmpdict["identifier"] = "icl_"+doi.split('/', 1)[-1]
            tmpdict["doi"] = doi
        elif "10." in eachitemcontent:
            doi = eachitemcontent
            tmpdict["identifier"] = "icl_"+doi.split('/', 1)[-1]
            tmpdict["doi"] = doi
    tmpdict['_id'] = tmpdict["identifier"]
    return(tmpdict)

Get the Author information

In [6]:
def get_authors(soupobject):
    authorsfield = soupobject.findAll("meta", {"name":"citation_author"})
    authors = get_meta_content(authorsfield)
    authorlist = []
    for eachauthor in authors:
        authparse = eachauthor.split(",")
        if (len(authparse) == 2) and len(authparse[1])<3:
            authdict = {'@type': 'outbreak:Person', 'affiliation': [], 'name': eachauthor, 
                       'familyName':authparse[0]}
        else:
            authdict = {'@type': 'outbreak:Person', 'affiliation': [], 'name': eachauthor}
        authorlist.append(authdict)
    return(authorlist)

Get the funding information

In [7]:
def generate_funding_dict(funder,identifier=None):
    fundict = {"@type": "MonetaryGrant",
               "funder": {"name": funder},
               "name": ""
               }    
    if identifier != None:
        fundict["identifier"]=identifier
    return(fundict)

def get_funding(soupobject):
    fundersfield = soupobject.findAll("meta", {"name":"DC.contributor"})
    funders = get_meta_content(fundersfield)
    fundercheck = len(fundersfield)
    if fundercheck > 0:
        identifiersfield = soupobject.findAll("meta", {"name":"DC.identifier"}) 
        fundidlist = []
        for eachitem in identifiersfield:
            eachitemcontent = eachitem.get("content")
            if ("https:" in eachitemcontent) or ("http:" in eachitemcontent):
                miscurls = eachitemcontent
            else:
                fundingid = eachitemcontent
                fundidlist.append(fundingid)
        fundlist = []
        i=0
        if len(funders)==len(fundidlist): ## There are the same amount of funders as ids
            while i < len(funders):
                fundict = generate_funding_dict(funders[i],fundidlist[i])
                fundlist.append(fundict)
                i=i+1
        elif len(funders)>len(fundidlist): ## There are more funders than ids, map the MR ones, then ignore ids
            mrfunds = [x for x in funders if "MRC" in x]
            mrids = [x for x in fundidlist if "MR" in x]
            while i < len(mrfunds):
                fundict = generate_funding_dict(mrfunds[i],mrids[i])
                fundlist.append(fundict)
                i=i+1
            remaining_funders = [x for x in funders if x not in mrfunds]
            remaining_fundids = [x for x in fundidlist if x not in mrids]
            j=0
            if (len(remaining_fundids)==0) and (len(remaining_funders)>0):
                while j<len(remaining_funders):
                    fundict = generate_funding_dict(remaining_funders[j])
                    fundlist.append(fundict)
                    j=j+1
        else: ##There are more ids than funders, and it will be impossible to map them
            while i < len(funders):
                fundict = generate_funding_dict(funders[i])
                fundlist.append(fundict)
                i=i+1            
        fundflag = True
    else:
        fundlist = []
        fundflag = False
    return(fundlist, fundflag)

## Main

In [14]:
reports_url = 'https://spiral.imperial.ac.uk/handle/10044/1/78555/simple-search?location=10044%2F1%2F78555&query=&filter_field_1=type&filter_type_1=equals&filter_value_1=Report&rpp=100&sort_by=dc.date.issued_dt&order=DESC&etal=0&submit_search=Update'
url_list = get_report_links(reports_url)
print(url_list)

['https://spiral.imperial.ac.uk/handle/10044/1/84879', 'https://spiral.imperial.ac.uk/handle/10044/1/84788', 'https://spiral.imperial.ac.uk/handle/10044/1/84283', 'https://spiral.imperial.ac.uk/handle/10044/1/84270', 'https://spiral.imperial.ac.uk/handle/10044/1/84220', 'https://spiral.imperial.ac.uk/handle/10044/1/84873', 'https://spiral.imperial.ac.uk/handle/10044/1/83928', 'https://spiral.imperial.ac.uk/handle/10044/1/83929', 'https://spiral.imperial.ac.uk/handle/10044/1/83545', 'https://spiral.imperial.ac.uk/handle/10044/1/82822', 'https://spiral.imperial.ac.uk/handle/10044/1/82551', 'https://spiral.imperial.ac.uk/handle/10044/1/82443', 'https://spiral.imperial.ac.uk/handle/10044/1/83126', 'https://spiral.imperial.ac.uk/handle/10044/1/80360', 'https://spiral.imperial.ac.uk/handle/10044/1/80295', 'https://spiral.imperial.ac.uk/handle/10044/1/79984', 'https://spiral.imperial.ac.uk/handle/10044/1/79837', 'https://spiral.imperial.ac.uk/handle/10044/1/79643', 'https://spiral.imperial.ac

In [9]:
#reports_url = 'https://spiral.imperial.ac.uk:8443/handle/10044/1/78555/simple-search?location=10044%2F1%2F78555&query=&filter_field_1=type&filter_type_1=equals&filter_value_1=Report&rpp=100&sort_by=score&order=DESC&etal=1&submit_search=Update'
reports_url = 'https://spiral.imperial.ac.uk/handle/10044/1/78555/simple-search?location=10044%2F1%2F78555&query=&filter_field_1=type&filter_type_1=equals&filter_value_1=Report&rpp=100&sort_by=dc.date.issued_dt&order=DESC&etal=0&submit_search=Update'
url_list = get_report_links(reports_url)
print("there are ",len(url_list)," urls to parse.")
curatedBy = create_curationObject()

## Pull the metadata from each report link and throw it into a dictionary
for each_url in url_list[0:2]:
    print("retrieving record for ",each_url)
    record_result = requests.get(each_url)
    print("parsing record")
    parsed_record = BeautifulSoup(record_result.text, "html.parser")
    print("transforming parsed record")
    base_info = transform_pub_meta(parsed_record)
    base_info["curatedBy"] = curatedBy
    print("fetching the authors")
    author_list = get_authors(parsed_record)
    print("fetching the funders")
    fund_list, fund_flag = get_funding(parsed_record)
    ## Create the Json
    print("creating the json file")
    base_info["author"] = author_list
    if fund_flag == True:
        base_info["funding"] = fund_list
    print("presenting the results ... ")
    print("\n")
    print(base_info)
    print("\n")
    print("starting next record ...")
    print("\n")


there are  10  urls to parse.
retrieving record for  https://spiral.imperial.ac.uk:8443//handle/10044/1/78707


ConnectionError: HTTPSConnectionPool(host='spiral.imperial.ac.uk', port=8443): Max retries exceeded with url: //handle/10044/1/78707 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000027EAD370BB0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:


fundersfield = parsed_record.findAll("meta", {"name":"DC.contributor"})
funders = get_meta_content(fundersfield)
fundercheck = len(fundersfield)
identifiersfield = parsed_record.findAll("meta", {"name":"DC.identifier"})
if fundercheck > 0:
    identifiersfield = parsed_record.findAll("meta", {"name":"DC.identifier"}) 
    fundidlist = []
    for eachitem in identifiersfield:
        eachitemcontent = eachitem.get("content")
        if ("https:" in eachitemcontent) or ("http:" in eachitemcontent):
            miscurls = eachitemcontent
        else:
            fundingid = eachitemcontent
            fundidlist.append(fundingid)
    fundlist = []
    i=0
    if len(funders)==len(fundidlist):
        while i < len(funders):
            fundict = generate_funding_dict(funders[i],fundidlist[i])
            fundlist.append(fundict)
            i=i+1
    elif len(funders)>len(fundidlist):
        mrfunds = [x for x in funders if "MRC" in x]
        mrids = [x for x in fundidlist if "MR" in x]
        while i < len(mrfunds):
            fundict = generate_funding_dict(mrfunds[i],mrids[i])
            fundlist.append(fundict)
            i=i+1
        remaining_funders = [x for x in funders if x not in mrfunds]
        remaining_fundids = [x for x in fundidlist if x not in mrids]
        print(remaining_funders,len(remaining_funders))
        print(remaining_fundids,len(remaining_fundids))
        j=0
        if (len(remaining_fundids)==0) and (len(remaining_funders)>0):
            while j<len(remaining_funders):
                fundict = generate_funding_dict(remaining_funders[j])
                fundlist.append(fundict)
                j=j+1                         
    fundflag = True
else:
    fundlist = []
    fundflag = False

In [ ]:
print(parsed_record)

In [ ]:
for each_url in url_list[1:2]:
    record_result = requests.get(each_url)
    parsed_record = BeautifulSoup(record_result.text, "html.parser")
    base_info = transform_pub_meta(parsed_record)
    base_info["curatedBy"] = curatedBy
    author_list = get_authors(parsed_record)
    print("fetching the funders")
    #fund_list, fund_flag = get_funding(parsed_record)
    fundersfield = parsed_record.findAll("meta", {"name":"DC.contributor"})
    funders = get_meta_content(fundersfield)
    fundercheck = len(fundersfield)
    if fundercheck > 0:
        identifiersfield = parsed_record.findAll("meta", {"name":"DC.identifier"}) 
        fundidlist = []
        for eachitem in identifiersfield:
            eachitemcontent = eachitem.get("content")
            if ("https:" in eachitemcontent) or ("http:" in eachitemcontent):
                miscurls = eachitemcontent
            else:
                fundingid = eachitemcontent
                fundidlist.append(fundingid)
        fundlist = []
        i=0
        while i < len(funders):
            fundict = {"@type": "MonetaryGrant",
                       "funder": {
                       "name": funders[i]
                       },
                      "identifier": fundidlist[i],
                      "name": ""
            }
            fundlist.append(fundict)
        fundflag = True
    else:
        fundlist = []
        fundflag = False
"""   
    ## Create the Json
    print("creating the json file")
    base_info["author"] = author_list
    if fund_flag == True:
        base_info["funding"] = fund_list
    print("presenting the results ... ")
    print("\n")
    print(base_info)

"""